# TMA4320 - Prosjekt 1 - Blind separasjon av signaler
***
Prosjekt skrevet av William Scott Grundeland Olsen, Gruppe 6. Levert inn 10.02.2019.
***
## Innholdsfortegnelse
- [Introduksjon.](#Introduksjon)
- [Importering av data.](#Importering_av_data)
- [Miksing av egenopplastede signaler.](#Miksing_av_egenopplastede_signaler)
- [Preprosessering.](#Preprosessering)
- [Maksimering av ikke-gaussiskhet.](#Maksimering_av_ikke-gaussiskhet)
- [Test av resultatene.](#Test_av_resultatene)
- [Oppsummering og vurdering av resultatene.](#Oppsummering_og_vurdering_av_resultatene)
- [Referanser.](#Referanser)
- [Kommentarer.](#Kommentarer)

<a id='Introduksjon'></a>

## Introduksjon

I dette prosjektet skal vi skrive et program i Python som kan blindt separere signaler, det vil si at både den såkalte miksematrisen og de separerte signalene er ukjente. Merk at gjennom dette prosjektet i Pythonkoden til vi bruke numpy arrays og ikke Python-lister. Dette vil gjøre komputasjonene enklere, og vil spare kjøretid.

Vi ser for oss at vi ønsker en rekke rene signaler $\mathbf{s}_j(t)$, for $j=1,2,\dots,n$ der $t\in[0,T]$ er lengden på signalene. Da vil signalet man registrerer i kilde $i$, $\mathbf{x}_i(t)$ være gitt ved

$$
\mathbf{x}_i(t)=a_{i1}\mathbf{s}_{1}(t)+a_{i2}\mathbf{s}_{2}(t)+\cdots+a_{in}\mathbf{s}_{n}(t).
$$

Her er både den såkalte miksematrisen $\mathbf{A}=((a_{ij}))$ og signalene $\mathbf{s}_j(t)$, ukjente. Det vi ønsker er da å løse likningen $\mathbf{s}=\mathbf{A}^{-1}\mathbf{x}$. For å løse denne må vi da finne en approksimasjon til matrisen $\mathbf{A}$, og det er i bunn og grunn dette vi skal gjøre gjennom prosjektet.

For at den uavhengige komponentanalysen (eller Independent Component Analysis, ICA) skal virke til å approksimere $\mathbf{A}$ må vi anta at kildene er statisktisk uavhengige, og at signalene er ikke-Gaussiske. Med andre ord skal ikke signalene være normalfordelte. For å få dette til vil vi maksimere ikke-Gaussiskheten til lineærkombinasjonen

$$
\mathbf{y}(t)=w_1\mathbf{x}_1+w_2\mathbf{x}_2+\cdots+\mathbf{x}_n.
$$

Da vi ikke kan endre på $\mathbf{x}_i$, så må vi maksimere med hesyn på $w_i$, for $i=1,2,\dots,n$. Denne maksimerte $\mathbf{y}$-en tas så som en approksimasjon til ett av signalene $\mathbf{s}_j$. Vi ønsker altså å finne $\mathbf{Z}_\text{ICA}$ som er matrisen som inneholder alle $\mathbf{s}_j$. Denne approksimasjonen vil vi utføre på for forskjellige måter senere i prosjektet, både ved kurtose og ved negentropy.

Vi vil først importere lydfilene som vi skal separere, og under implementeres algoritmen med funksjonene som trengs for å utføre de numeriske beregningene. Jeg har importert noen ekstra lydklipp som lastes opp og mikses sammen for å teste algoritmen på. Det følger med en forklaring til hver kodeblokk for å kunne forstå hva som implementeres hvor. Prosjektet avsluttes med en test av signalene før en oppsummering og virdering av resultatene.

<a id='Importering_av_data'></a>

## Importering av data

Vi importerer først de bibliotekene vi vil få bruk for i dette prosjektet ved koden under.

In [1]:
import numpy as np
from wav_file_loader import read_wavefiles
import IPython.display as ipd
import warnings

Her importeres lydfilene som allerede fulgte med oppgaven. Dersom lydfilene ligger på audio/mix_1.wav, audio/mix_2.wav og audio/mix_3.wav, kan de tre miksede lydklippene som vi skal bruke å teste ut algoritmen på importeres av følgende kode:

In [2]:
paths = ['audio/mix_1.wav', 'audio/mix_2.wav', 'audio/mix_3.wav']
data, sampling_rate = read_wavefiles(paths)
num_signals = data.shape[0]

For at de tre miksede lydsignalene skal få omtrent samme lydstyrke, normaliseres lydsignalets volum i følgende kode. Her skaleres amplitudene slik at maksimum av `data[i]`, der `data` er inputmatrisen, er 1.

In [3]:
def normalize_audio(data):
    abs_data = np.absolute(data)
    maximums = np.amax(abs_data, 1)
    # Divide each row by a different vector element:
    data = data / maximums.reshape((3, 1))
    return data

data = normalize_audio(data)

Med følgende kode kan man spille av de tre lydklippene:

In [4]:
ipd.display(ipd.Audio(data[0,:], rate=sampling_rate))
ipd.display(ipd.Audio(data[1,:], rate=sampling_rate))
ipd.display(ipd.Audio(data[2,:], rate=sampling_rate))

Under laster vi opp tre lydklipp til, som ligger på audio/tired.wav, audio/trillions.wav og audio/kitchen.wav. Disse skal mikses, for så å separeres. Her finnes også antall signaler, `num_signals_me`, samplingsraten, `sampling_rate_me`, og dataen til signalene i matriseform, `data_me`.

In [5]:
paths_me = ['audio/tired.wav', 'audio/trillions.wav', 'audio/kitchen.wav']
data_me, sampling_rate_me = read_wavefiles(paths_me)
num_signals_me = data_me.shape[0]
data_me = normalize_audio(data_me)

I cellen under kan de tre lydklippene spilles av slik de var når de ble lastet opp. Detter er for å kunne sammenlikne kvaliteten med de separerte signalene.

In [6]:
ipd.display(ipd.Audio(data_me[0,:], rate=sampling_rate_me))
ipd.display(ipd.Audio(data_me[1,:], rate=sampling_rate_me))
ipd.display(ipd.Audio(data_me[2,:], rate=sampling_rate_me))

For å vise at det er forskjell på hvor godt algoritmen virker basert på lydfilene som lastes opp, laster vi nå opp tre lydklipp som ligger på audio/tired.wav, audio/good_morning.wav og audio/kitchen.wav. Forskjellen er altså at én av filene er byttet ut fra tilfellet over. Vi kaller disse for egenopplastede signaler nummer 2, i motsetning til nummer 1 over. Under laster vi dem opp, og spiller dem av på tilsvarende måte som over.

In [7]:
paths_me_2 = ['audio/tired.wav', 'audio/good_morning.wav', 'audio/kitchen.wav']
data_me_2, sampling_rate_me_2 = read_wavefiles(paths_me_2)
num_signals_me_2 = data_me_2.shape[0]
data_me_2 = normalize_audio(data_me_2)

In [8]:
ipd.display(ipd.Audio(data_me_2[0,:], rate=sampling_rate_me_2))
ipd.display(ipd.Audio(data_me_2[1,:], rate=sampling_rate_me_2))
ipd.display(ipd.Audio(data_me_2[2,:], rate=sampling_rate_me_2))

<a id='Miksing_av_egenopplastede_signaler'></a>

## Miksing av egenopplastede signaler

I dette avsnittet defineres funksjonene `normalize_rowsums` og `random_mixing_matrix` som muliggjør å mikse signaler til én. Dette gjøres med de tre ekstra lydklippene over.

I følgende funksjon divideres hvert element i radene i inputmatrisen `A` med summen av samme rad. Dette normaliserer hver rad, slik at summen over radene er 1. Altså dersom $\mathbf{x}_j(t_i)$ er rad $j$, for $i=1,2,\dots,N$, så vil denne erstattes med

$$
\mathbf{x}_j(t_i)\cdot\left[\sum_{n=1}^N\mathbf{x}_j(t_n)\right]^{-1},
$$

ettersom rad $j$ har elementene $\mathbf{x}_j(t_1),\mathbf{x}_j(t_2),\dots,\mathbf{x}_j(t_N)$.

In [9]:
def normalize_rowsums(A):
    the_sum = np.sum(A, 1)
    A = A / the_sum.reshape((len(A), 1))
    return A

Funksjonen under lager en tilfeldig matrise `A` som returneres sammen med bruk av funksjonen over. Elementene i matrisen er små positive nummer som ikke er for nærme null. Funksjonen tar inn antall signaler, `signals`, og antall observasjoner `observations`.

In [10]:
def random_mixing_matrix(signals, observations):
    A = 0.25 + np.random.rand(observations, signals)
    return normalize_rowsums(A)

Følgende kodeblokk lages den tilfeldige matrisen `A`, og deretter den miksede dataen `data_mixed`.

In [11]:
A = random_mixing_matrix(num_signals_me, num_signals_me)
data_mixed = normalize_audio(A @ data_me)

Under er det mulig å spille av de miksede signalene av de tre ekstra lydklippene.

In [12]:
ipd.display(ipd.Audio(data_mixed[0,:], rate=sampling_rate))
ipd.display(ipd.Audio(data_mixed[1,:], rate=sampling_rate))
ipd.display(ipd.Audio(data_mixed[2,:], rate=sampling_rate))

Tilsvarede mikser vi de egenopplastede nummer 2 filene ved å lage `A_2` og deretter `data_mixed_2`, og spiller dem av.

In [13]:
A_2 = random_mixing_matrix(num_signals_me_2, num_signals_me_2)
data_mixed_2 = normalize_audio(A_2 @ data_me_2)

In [14]:
ipd.display(ipd.Audio(data_mixed_2[0,:], rate=sampling_rate))
ipd.display(ipd.Audio(data_mixed_2[1,:], rate=sampling_rate))
ipd.display(ipd.Audio(data_mixed_2[2,:], rate=sampling_rate))

<a id='Preprosessering'></a>

## Preprosessering

I dette avsnittet skrives funksjonene `center_rows` og `whiten_rows` som opererer på matrisen `Z`. Denne er et array av dimensjon $d\times N$ av miksede signaler.

Følgende funksjon, `center_rows`, endrer matrisen `Z` slik at hver rad får null i gjennomsnitt. Funksjonen tar inn en vilkårlig matrise `Z`, og for hver verdi i hver rad i matrisen subtraheres gjennomsnittet av alle verdiene til den tilhørende raden. Altså, dersom $\mathbf{x}_j(t_i)$ er rad $j$ for $i=1,2,\dots,N$, så vil den sentrerte raden være gitt ved

$$
\mathbf{x}_j(t_i)-\frac{1}{N}\sum_{n=1}^N\mathbf{x}_j(t_n),
$$

for $i=1,2,\dots,N$. Da endres `Z` til en ny sentrert matrise som returneres. Det var oppgitt at en variabel `mus` skulle returneres, men denne brukes ikke videre, og unnlates da å returneres da jeg ikke definerer den til å begynne med.

In [15]:
def center_rows(Z):
    return Z - np.mean(Z, axis=1).reshape(len(Z), 1)

Følgende funksjon, `whiten_rows`, tar inn en matrise `Z`. Den regner ut kovariansmatrisen `C` til `Z`, og komputerer den inverse kvadratroten til `C`, kalt `inv_sqrt_C`. Denne beregnes ved at $\mathbf{C}^{-1/2}=\mathbf{E}\cdot\mathbf{\Lambda}^{-1/2}\cdot\mathbf{E}^\top$, der $\mathbf{E}$ er matrisen med egenvektorer, mens $\mathbf{\Lambda}$ har egenverdiene langs diagonalen. Deretter vil matriseprodunktet $\mathbf{C}^{-1/2}\cdot\mathbf{Z}$ gi den ønskede `Z_whitened`. Denne returneres. Jeg valgte å ikke returnere den inverse kvadratroten til kovariansmatrisen, selv om dette stod i beskrivelsen, da den ikke har en hensikt videre i programmeringen.

In [16]:
def whiten_rows(Z):
    C = np.cov(Z)
    E, Lambda, _ = np.linalg.svd(C, full_matrices=False)
    inv_sqrt_C = E @ np.diag(1 / np.sqrt(Lambda)) @ E.T
    Z_whitened = inv_sqrt_C @ Z
    return Z_whitened

Når vi har utført stegene beskrevet over så har dataene vi tidligere kalte $\mathbf{x}$ blitt transformert til noe vi kan kalle $\tilde{\mathbf{x}}$, der det enda gjelder at $\mathbf{s}=\tilde{\mathbf{A}}^{-1}\mathbf{x}$. Nå har derimot $\tilde{\mathbf{A}}$, og dermed $\tilde{\mathbf{A}}^{-1}$, blitt ortogonal i motsetning til $\mathbf{A}$.

<a id='Maksimering_av_ikke-gaussiskhet'></a>

## Maksimering av ikke-gaussiskhet

I dette avsnittet skrives funksjonene `normalize_rownorms`, `decorrelate_weights`, `measure_of_convergence`, og `update_W` og `fast_ICA` som kommer i to variasjoner; én for kurtose, og én for negentropy.

Funksjonen `normalize_rownorms` tar inn en matrise `Z` og finner den Euklidske normen til hver rad. Hvert element i tilhørende rad divideres på denne normen. Altså vil rad $j$, symbolisert ved $\mathbf{x}_j$ bli omgjort til raden $\mathbf{x}_j/||\mathbf{x}_j||$. Dette endrer matrisen `Z` og denne returneres.

In [17]:
def normalize_rownorms(Z):
    return Z / np.linalg.norm(Z, axis=1).reshape(len(Z), 1)

Funksjonen `decorrelate_weights` tar inn en $d\times d$-matrise `W` som projiseres til en ortogonal matrise `W_decorrelated` gitt ved

$$
\mathbf{W}_\text{decorrelated}=\left(\mathbf{W}\cdot\mathbf{W}^\top\right)^{-1/2}\cdot\mathbf{W},
$$

som returneres. Her har vi brukt faktorisering av $\mathbf{W}\cdot\mathbf{W}^\top$ i matriser av dens egenverdier og egenvektorer, $\mathbf{W}\cdot\mathbf{W}^\top=\mathbf{E}\cdot\mathbf{\Lambda}\cdot\mathbf{E}^\top$ for å finne dens inverse kvadratrot. Her er $\mathbf{E}$ matrisen med egenvektorer, mens $\mathbf{\Lambda}$ har egenverdiene langs diagonalen.

In [18]:
def decorrelate_weights(W):
    W_transpose = W.T
    WWT = W @ W_transpose
    E, Lambda, _ = np.linalg.svd(WWT, full_matrices=False)
    W_decorrelated = E @ np.diag(1 / np.sqrt(Lambda)) @ E.T @ W
    return W_decorrelated

Jeg har valgt å lage to funksjoner som oppdaterer `W`, der den ene bruker tilfellet med kutrose, og den andre med negentropy.

Følgende funksjon, `update_W_kurtosis`, tar inn en $d\times d$-matrise `W`, og den sentrerte "whitened" dataen i $d\times N$-matrisen `Zcw`. De anonyme funksjonene `G` og `Gp`, den deriverte av `G`, defineres som for kurtose, $G(u)=4u^3$ og dermed $G'(u)=12u^2$, og anvendes på hvert element i matrisen `s_k`. Dette gir oss henholdsvis `bold_G` og `bold_Gp`, som begge er matriser. Deretter beregnes `W_pluss_normalized`, der alle radene er normalisert til én, ved at radene til matrisen

$$
\mathbf{W}^+=\frac{1}{N}\cdot\mathbf{G}\cdot\mathbf{Z}^\top_\text{cw} - \text{diag}\left(E[\mathbf{G}']\right)\cdot\mathbf{W}
$$

normaliseres. Her er $N$ tilsvarende `len(bold_G[0])`, mens de andre variablene er tilsvarede som i koden under. Funksjonen `decorrelate_weights` kalles så med `W_pluss_normalized` som input. Det er denne verdien som returneres

Der vi har definerte de anonyme funksjonene kan dette føre til noe lengre kjøretid. I dette prosjektet derimot føler jeg at denne økingen er neglisjerbar, og føler at de øker lesbarheten i koden, i hvert fall i tilfellet for `update_W_negentropy`. Man kunne derimot for eksempel latt `bold_G = 4 * s_k**3` og tilsvarende.

In [19]:
def update_W_kurtosis(W, Zcw):
    G = lambda u: 4 * u**3
    G_prime = lambda u: 12 * u**2
    s_k = W @ Zcw
    bold_G = G(s_k)
    bold_G_prime = G_prime(s_k)
    ZcwT = Zcw.T
    EG_prime = np.mean(bold_G_prime, axis=1)
    diag_EG_prime = np.diag(EG_prime)
    W_pluss_normalized = normalize_rownorms(bold_G @ ZcwT / len(bold_G[0]) - diag_EG_prime @ W)
    return decorrelate_weights(W_pluss_normalized)

Funksjonen `update_W_negentropy` gjør det tilsvarende som `update_W_kurtosis`, men her definerer vi $G(u)=u\text{e}^{-u^2/2}$, slik at den deriverte blir $G'(u)=\text{e}^{-u^2/2}-u^2\text{e}^{-u^2/2}$. Ellers har de samme input- og returparametre.

In [20]:
def update_W_negentropy(W, Zcw):
    G = lambda u: u * np.exp(-u**2 / 2)
    G_prime = lambda u: np.exp(-u**2 / 2) * (1 - u**2)
    s_k = W @ Zcw
    bold_G = G(s_k)
    bold_G_prime = G_prime(s_k)
    ZcwT = Zcw.T
    EG_prime = np.mean(bold_G_prime, axis=1)
    diag_EG_prime = np.diag(EG_prime)
    W_pluss_normalized = normalize_rownorms(bold_G @ ZcwT / len(bold_G[0]) - diag_EG_prime @ W)
    return decorrelate_weights(W_pluss_normalized)

Funksjonen `measure_of_convergence` regner ut og returnerer konvergenskriteriet $\delta$ som er gitt ved

$$
\delta = \max_{1\leq i\leq d}\left(1-\left|\sum_{j=1}^d(\mathbf{W}_k)_{ij}(\mathbf{W}_{k-1})_{ij}\right|\right).
$$

I koden vil `W1` represenere $\mathbf{W}_{k-1}$ og `W2` representerer $\mathbf{W}_k$.

In [21]:
def measure_of_convergence(W1, W2):
    return np.amax(1 - np.absolute(np.sum(W2 * W1, axis=1)))

Følgende funksjon, `fast_ICA_kurtosis`, og funksjonen `fast_ICA_negentropy` definert under gjør samme operasjoner. Forskjellen mellom dem er at de bruker hennholdsvis $G$-funksjonene som representerer kurtose og negentropy. Dermed vil `fast_ICA_kurtosis` anvende `update_W_kurtosis`, mens `fast_ICA_negentropy` anvender `update_W_negentropy`.

Først finnes `Z_centered_whitened` fra `Z` ved å bruke funksjoner allerede definert. Deretter initialiseres `W_0`, `delta` og `number_of_iterations` til henholdsvis en tilfeldig matrise av dimensjoner `signals_to_find`$\times$`signals_to_find`, uendelig og null. Dette er for å starte iterasjonsløkken i tilfellet for `delta` og `number_of_iterations`, og for å gi oss en matrise å oppdatere ved `update_w` for enten kurtose eller negentropy for tilfellet med `W_0`. Deretter initialiseres `while`-løkken der `number_of_iterations` øker med én, og vi finner en ny `W` fra `W_0` og `Z_centered_whitened`. `delta` oppdateres ved hjelp av `W_0` og den nye `W`. Ettersom vi vil finne en ny verdi for `W` neste iterasjon setter vi `W_0 = W`. Etter enten antall iterasjoner er oppnådd eller toleransen er tilfredstillt stopper `while`-løkken. Som defult-verdi er toleransen satt til 1e-10 og maks antall iterasjoner er satt til 100. Når `while`-løkken stopper vil `Z_ICA` som er matriseproduktet av den endelige `W` og `Z_centered_whitened` returneres. I tillegg returneres `number_of_iterations` for å senere kunne teste hvor ofte funksjonene feiler for gitte signaler. Jeg valgte å ikke returnere `W` som beskrevet i oppgaven, da jeg ikke burker denne videre.

In [22]:
def fast_ICA_kurtosis(Z, signals_to_find, tol=1e-10, max_iter=100):
    Z_centered_whitened = whiten_rows(center_rows(Z))
    W_0 = normalize_rownorms(np.random.rand(signals_to_find, signals_to_find))
    delta, number_of_iterations = np.inf, 0
    while (delta > tol) and (number_of_iterations < max_iter):
        number_of_iterations += 1
        W = update_W_kurtosis(W_0, Z_centered_whitened)
        delta = measure_of_convergence(W_0, W)
        W_0 = W
    if number_of_iterations == max_iter:
        warnings.warn('The number of iterations reached the maximum number of iterations. \
The approximation may deviate from the desired result.')
    Z_ICA = W @ Z_centered_whitened
    return Z_ICA, number_of_iterations

In [23]:
def fast_ICA_negentropy(Z, signals_to_find, tol=1e-10, max_iter=100):
    Z_centered_whitened = whiten_rows(center_rows(Z))
    W_0 = normalize_rownorms(np.random.rand(signals_to_find, signals_to_find))
    delta, number_of_iterations = np.inf, 0
    while (delta > tol) and (number_of_iterations < max_iter):
        number_of_iterations += 1
        W = update_W_negentropy(W_0, Z_centered_whitened)
        delta = measure_of_convergence(W_0, W)
        W_0 = W
    if number_of_iterations == max_iter:
        warnings.warn('The number of iterations reached the maximum number of iterations. \
The approximation may deviate from the desired result.')
    Z_ICA = W @ Z_centered_whitened
    return Z_ICA, number_of_iterations

Vi har nå funnet en approksimasjon til det vi kalte $\tilde{\mathbf{A}}^{-1}$ i forige avsnitt. Denne approksimasjonen har vi kalt `W` her, og da er $\mathbf{W}\approx\tilde{\mathbf{A}}^{-1}$, slik at vår approksimasjon til kildesignalene blir $\mathbf{Z}_\text{ICA}$. Det er denne approksimasjonen vi tester i neste avsnitt.

<a id='Test_av_resultatene'></a>

## Test av resultatene

Under følger test av de vedlagte lydklippene ved henholdsvis kurtose og negentropy.

In [24]:
Z_ICA_kurtosis, _ = fast_ICA_kurtosis(data, num_signals)
Z_ICA_kurtosis = normalize_audio(Z_ICA_kurtosis)

ipd.display(ipd.Audio(Z_ICA_kurtosis[0,:], rate=sampling_rate))
ipd.display(ipd.Audio(Z_ICA_kurtosis[1,:], rate=sampling_rate))
ipd.display(ipd.Audio(Z_ICA_kurtosis[2,:], rate=sampling_rate))

In [25]:
Z_ICA_negentropy, _ = fast_ICA_negentropy(data, num_signals)
Z_ICA_negentropy = normalize_audio(Z_ICA_negentropy)

ipd.display(ipd.Audio(Z_ICA_negentropy[0,:], rate=sampling_rate))
ipd.display(ipd.Audio(Z_ICA_negentropy[1,:], rate=sampling_rate))
ipd.display(ipd.Audio(Z_ICA_negentropy[2,:], rate=sampling_rate))

I cellen under tester vi hvor mange ganger av 1000 der kurtose og negentropy feiler for signalene gitt i oppgaven. Dette vil kunne varriere for hver gang koden kjøres, da det avhenger av den tilfeldige matrisen `W_0` som initialiseres hver gang koden kjøres. Derimot viser kjøringer at det ikke er stor variasjon. Merk at å kjøre koden vil ta en liten stund da den kjører gjennom prosessene for kurtose og negentropy 1000 ganger.

In [26]:
counter_kurtosis, counter_negentropy = 0, 0
for _ in range(1000):
    _, num_it_kurtosis = fast_ICA_kurtosis(data, num_signals)
    _, num_it_negentropy = fast_ICA_negentropy(data, num_signals)
    if num_it_kurtosis == 100:
        counter_kurtosis += 1
    if num_it_negentropy == 100:
        counter_negentropy += 1
print('Kurtose:', counter_kurtosis, '\nNegentropy:', counter_negentropy)

Kurtose: 0 
Negentropy: 0


De to følgende kodeblokkene gjengir de signalene jeg selv lastet opp og mikset for henholdsvis kurtose og negentropy. Dette er de egenopplastede filene nummer 1.

In [27]:
Z_ICA_kurtosis_me, _ = fast_ICA_kurtosis(data_mixed, num_signals_me)
Z_ICA_kurtosis_me = normalize_audio(Z_ICA_kurtosis_me)

ipd.display(ipd.Audio(Z_ICA_kurtosis_me[0,:], rate=sampling_rate))
ipd.display(ipd.Audio(Z_ICA_kurtosis_me[1,:], rate=sampling_rate))
ipd.display(ipd.Audio(Z_ICA_kurtosis_me[2,:], rate=sampling_rate))

In [28]:
Z_ICA_negentropy_me, _ = fast_ICA_negentropy(data_mixed, num_signals_me)
Z_ICA_negentropy_me = normalize_audio(Z_ICA_negentropy_me)

ipd.display(ipd.Audio(Z_ICA_negentropy_me[0,:], rate=sampling_rate))
ipd.display(ipd.Audio(Z_ICA_negentropy_me[1,:], rate=sampling_rate))
ipd.display(ipd.Audio(Z_ICA_negentropy_me[2,:], rate=sampling_rate))

I cellen under tester vi hvor mange ganger av 1000 der kurtose og negentropy feiler for de egenopplastede lydfilene. Det vil si hvor mange ganger vi når maks antall iterasjoner på 100. Dette vil varriere for hver gang koden kjøres, da det avhenger av den tilfeldige matrisen `W_0` som initialiseres hver gang koden kjøres. Merk at å kjøre koden vil ta en liten stund da den kjører gjennom prosessene for kurtose og negentropy 1000 ganger.

In [29]:
counter_kurtosis, counter_negentropy = 0, 0
for _ in range(1000):
    _, num_it_kurtosis = fast_ICA_kurtosis(data_mixed, num_signals_me)
    _, num_it_negentropy = fast_ICA_negentropy(data_mixed, num_signals_me)
    if num_it_kurtosis == 100:
        counter_kurtosis += 1
    if num_it_negentropy == 100:
        counter_negentropy += 1
print('Kurtose:', counter_kurtosis, '\nNegentropy:', counter_negentropy)

Kurtose: 0 
Negentropy: 0


Til slutt gjengis de egenopplastede signalene nummer 2, som separert, ved henholdsvid kurtose og negentropy. Merk at du kanskje må kjøre koden noen ganger for at det skal konvergere, og toleransen nås innen maks antall iterasjoner er oppnådd.

In [30]:
Z_ICA_kurtosis_me_2, _ = fast_ICA_kurtosis(data_mixed_2, num_signals_me_2)
Z_ICA_kurtosis_me_2 = normalize_audio(Z_ICA_kurtosis_me_2)

ipd.display(ipd.Audio(Z_ICA_kurtosis_me_2[0,:], rate=sampling_rate))
ipd.display(ipd.Audio(Z_ICA_kurtosis_me_2[1,:], rate=sampling_rate))
ipd.display(ipd.Audio(Z_ICA_kurtosis_me_2[2,:], rate=sampling_rate))

In [31]:
Z_ICA_negentropy_me_2, _ = fast_ICA_negentropy(data_mixed_2, num_signals_me_2)
Z_ICA_negentropy_me_2 = normalize_audio(Z_ICA_negentropy_me_2)

ipd.display(ipd.Audio(Z_ICA_negentropy_me_2[0,:], rate=sampling_rate))
ipd.display(ipd.Audio(Z_ICA_negentropy_me_2[1,:], rate=sampling_rate))
ipd.display(ipd.Audio(Z_ICA_negentropy_me_2[2,:], rate=sampling_rate))

I cellen under tester vi hvor mange ganger av 1000 der kurtose og negentropy feiler for de egenopplastede lydfilene. Det vil si hvor mange ganger vi når maks antall iterasjoner på 100. Dette vil varriere for hver gang koden kjøres, da det avhenger av den tilfeldige matrisen `W_0` som initialiseres hver gang koden kjøres. Merk at å kjøre koden vil ta en liten stund da den kjører gjennom prosessene for kurtose og negentropy 1000 ganger. Du kan se bort fra advarselen som kastes, da dette gjenspeiles i tallene som printes.

In [32]:
counter_kurtosis, counter_negentropy = 0, 0
for _ in range(1000):
    _, num_it_kurtosis = fast_ICA_kurtosis(data_mixed_2, num_signals_me_2)
    _, num_it_negentropy = fast_ICA_negentropy(data_mixed_2, num_signals_me_2)
    if num_it_kurtosis == 100:
        counter_kurtosis += 1
    if num_it_negentropy == 100:
        counter_negentropy += 1
print('Kurtose:', counter_kurtosis, '\nNegentropy:', counter_negentropy)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: The number of iterations reached the maximum number of iterations. The approximation may deviate from the desired result.
  if sys.path[0] == '':


Kurtose: 300 
Negentropy: 182


<a id='Oppsummering_og_vurdering_av_resultatene'></a>

## Oppsummering og vurdering av resultatene

Fra testene i forige avsnitt legger vi merke til at både `Z_ICA_kurtosis` og `Z_ICA_negentropy` separerer signalene, men de virker bedre for signalene som fulgte med oppgaven. Dette kan forklares med at forskjellen mellom de tre signalene som fulgte med er større enn for de egenopplastede signalene. Det vil si at i signalene som fulgte med er det en mannsstemme, en kvinnestemme, og en melodi, mens i de egenopplastede signalene er det kun mannsstemmer. Ut i fra lydkvaliteten merker vi at forskjellem mellom dem er neglisjerbar for signalene gitt i oppgaven og de egenopplastede signalene nummer 1. Ved noen kjøringer kan det høres en forskjell der enten kurtose eller negentropy er best, men disse er neglisjerbare i helhet, i hvert fall i mine øre. Det er klart at de egenopplastede signalene nummer 2 ikke presterer like godt som nummer 1 eller signalene som fulgte med i oppgaven. Dette kan, i tillegg til bare mannsstemmer, forklares med at én av de tre lydsignalene er særdeles høyere enn de andre to. Dette gjør det vanskeligere for algoritmen å oppfatte de to resterende signalene, og vi kan høre at særlig ett av dem sliter. Vi hører at i to av avspillingene er det "Good morning Vietnam" som er mest tydlig. Det varrierer derimot hvilke av de to andre som man kan høre tydelig, og det varrierer gjerne også med tanke på om det er kurtose eller negentropy.

Fra koden i forrige avsnitt kan vi oppsummere resultatene av antall ganger de forskjellige versjonene og de forskjellige lydklippene feiler når de blir kjørt 1000 ganger i en tabell:

| Feilprosent | Gitt i oppgaven | Egenopplastet 1 | Egenopplastet 2 |
| --- | --- | --- | --- |
| **Kurtose** | 0 % | 0 % | 30.0 % |
| **Negentropy** | 0 % | 0 % | 18.2 % |

Merk at disse resultatene er gitt med toleranse 1e-10 og maksimalt antall iterasjoner på 100. Dette vil si at når kode kjøres 1000 ganger for både kurtose og negentropy, og for de gitte signalene og de egenopplastede signalene, nås toleranseverdien på mindre enn 100 iterasjoner. Disse tallene kan naturligvis endres hvis koden kjøres på nytt. Men over 1000 iterasjoner ligger det ganske jevnt på verdiene i tabellen. Det er størst usikkerhet i tallene for de egenopplastede signalene nummer 2, som kan varriere endel, da de to andre har ligget på 0 % ved hver kjøring av koden.

Man kunne også kjørt disse løkkene flere ganger og funnet gjennomsnittet over flere iterasjoner, men min erfaring viser at det ligger jevnt når man først itererer 1000 ganger. Det ville også tatt mye lenger tid å funnet gjennomsnittet av kjøringer som allerede iterers 1000 ganger.

Oppsummert legger vi merke til at algoritmen gjør sin jobb og separerer signalene, i to av tilfellene, og den virker best på signalene inkludert i oppgaven. Som hintet til tidligere kan dette følge av at alle tre signalene i de egenopplastede signalene nummer 1 er veldig like hverandre i forhold til signalene i oppgaven. Derfor kan det være vanskeligere for algoritmen å separere disse. Man kan også høre at for de egenopplastede signalene er litt utvidet da de har blitt separert, eller at lydklippet er blitt lenger, og litt utdradd. De er uansett separert, slik at det er mulig å høre en god likhet mellom de originale signalene og de separerte signalene. For de egenopplastede signalene nummer 2 separeres de nok ikke godt nok på grunn av den høylytte naturen til ett av klippene. Dette overdøver de andre, og separasjonen blir ikke helt tilfredstillende. Man kan derimot uansett høre en viss separasjon, som tilsier at algoritmen fungerer, bare ikke optimalt for alle lydsignaler. 

<a id='Referanser'></a>

## Referanser

Gjennom prosjektet er det brukt
- prosjektbeskrivelsen tilgjengelig på Blackborad, og
- A. Hyvärinen og E. Oja, *Independent component analysis: algorithms and applications*, Neural networks 13 (2000) 411–430.

<a id='Kommentarer'></a>

## Kommentarer
### Prosjektbeskrivelse
* Veldig god prosjektbeskrivelse. Du forklarer godt hva prosjektet går ut på.
* Modellen for prosjektet blir til noen grad diskutert, men dette kan med fordel diskuteres grundigere.
* Det er en fin beskrivelse av implementasjonen.

### Kode
* Koden er ryddig og oversiktlig.
* Veldig godt dokumentert.
* Det er fint at både negentropi og kurtose er undersøkt.
* Bra med egne lydfiler.

### Diskusjon og konklusjon
* Jeg liker svært godt fremgangsmåten for å beskrive hvilke `W_0` som fungerer.
* Kan gjerne gå mer i dybden på modellen.
* Kunne gjerne diskutert kjøretid for prosjektet.

### Poengsum
Poengsummen for oppgaven ble 9.5, men på grunn av bonuspoeng blir det totalt 10 poeng.